# Import Modules

In [1]:
# Setting Custom Paths ********************************************************
# *****************************************************************************
import os
import sys

sys.path.insert(0, os.path.join(
    os.environ["PROJ_irox"],
    "data"))

# Python Modules **************************************************************
# *****************************************************************************
import copy
import csv

import plotly.plotly as py
# import plotly.graph_objs as go
import plotly.io as pio

import pandas as pd

# My Modules ******************************************************************
# *****************************************************************************

# Local Imports ***************************************************************
# *****************************************************************************
# from parse_plot_data import get_plot_data
from methods import process_data_for_plot, get_layout

from proj_data_irox import (
    proj_dir_name,
    )

# Script Inputs

In [2]:
save_plot = True


compenv_var = os.environ.get("COMPENV", None)
if os.environ.get("COMPENV", None) is not None:
    if compenv_var == "wsl":
        orca_avail = False
    else:
        orca_avail = True
else:
    orca_avail = True

# Reading Data

## 1st Generation

In [3]:
file_path_i = os.path.join(
    os.environ["PROJ_irox"],
    "workflow/ml_modelling/ml_convergence_plots",
    "chris_data_for_figures/iro2/new_190329",
    "OQMD_IrO2___published_IrO2.txt",
    )

with open(file_path_i, "r") as fle:
    data = csv.reader(fle)
    data_list = []
    for i in data:
        i = i[0].split("\t")
        row_i = {
            "id": int(i[0]),
            "form_e_0": float(i[1]),
            "uncert_0": float(i[2]),
            }

        data_list.append(row_i)

df1 = pd.DataFrame(data_list)
df1["ind"] = [i for i in range(len(df1))]

## 2nd Generation

In [4]:
file_path_i = os.path.join(
    os.environ["PROJ_irox"],
    "workflow/ml_modelling/ml_convergence_plots",
    "chris_data_for_figures/iro2/new_190329",
    "IrO2_after_active_learning_DFT.txt",
    )

with open(file_path_i, "r") as fle:
    data = csv.reader(fle)
    data_list = []
    for i in data:
        i = i[0].split("\t")
        row_i = {
            "id": int(i[0]),
            "form_e_0": float(i[1]),
            "uncert_0": float(i[2]),
            }

        data_list.append(row_i)

df2 = pd.DataFrame(data_list)
df2["ind"] = [i for i in range(len(df1))]

# Processing Data

In [5]:
# list_0 = df1["form_e_0"].tolist()
# for i_ind, i in enumerate(list_0):
#     tmp = 42

#     print(i - list_0[i_ind - 1])

In [6]:
# Generation to be the baseline for x-axis ordering
# This gen will have a continious color scale gradient
gen_ordered = "round_1"

if gen_ordered == "round_2":
    gen_other = "round_1"
elif gen_ordered == "round_1":
    gen_other = "round_2"

new_col = list(df1["ind"].astype(float) / float(df1["ind"].max()))

df1["color_order"] = new_col


# df["iro3"]

id_color_grad_mapping = dict(zip(
    df1["id"],
    df1["color_order"],
    ))


def struct_mapping(row_i):
    """
    """
    order_i = id_color_grad_mapping[row_i["id"]]

    return(order_i)

df2["color_order"] = df2.apply(
    struct_mapping,
    axis=1,
    )

# .sort_index(axis=1, inplace=True)

In [7]:
data_final = []
data_i = process_data_for_plot(
    df1,
    color0='rgba(30,40,50,0.5)',
    color1='rgba(30,40,50,0.5)',
    name='1st_gen',
    marker_color_mode='array',
    energy_key='form_e_0',
    marker_color_key='color_order',
    uncertainty_key='uncert_0',
    text_key='id',
    process_text_for_hover=False,
    explicit_error_bars=True,
    filled_error_traces=False,
    )
data_final += data_i



data_final2 = []
data_i = process_data_for_plot(
    df2,
    color0='rgba(90,0,130,0.5)',
    color1='rgba(90,0,130,0.5)',
    name='2nd_gen',
    marker_color_mode='array',
    energy_key='form_e_0',
    marker_color_key='color_order',
    uncertainty_key='uncert_0',
    text_key='id',
    process_text_for_hover=False,
    explicit_error_bars=True,
    filled_error_traces=False,
    )
data_final2 += data_i


# data_final += data_i

# Plotting

In [8]:
if save_plot:
    save_dir = proj_dir_name
else:
    save_dir = "__temp__"

if not os.path.exists('images'):
    os.mkdir('images')

# layout = get_layout()
# layout["showlegend"] = False
# layout["xaxis"]["range"] = [0, 700]
# layout["yaxis"]["range"] = [-1.5, 2.5]

# layout["width"] = 1.05 * 18.7 * 37.795275591
# layout["height"] = 0.4 * 18.7 * 37.795275591

## Configure Plot Layout

In [9]:
layout = get_layout()

layout["yaxis"]["range"] = [-1.5, 2.5]
layout["yaxis"]["dtick"] = 0.5
layout["yaxis"]["title"] = ""

layout["xaxis"]["title"] = ""
layout["xaxis"]["range"] = [0, 700]
layout["xaxis"]["showticklabels"] = False

layout["width"] = 1.05 * 18.7 * 37.795275591
layout["height"] = 0.4 * 18.7 * 37.795275591
layout["showlegend"] = False

layout["font"] = dict(
    family='Arial',
    size=10,
    color='black',
    )

# layout["shapes"] = [
#     {
#         "type": "rect",
#         "xref": "x",
#         "yref": "y",
#         "x0": zoom_in_ranges["x"][0],
#         "y0": zoom_in_ranges["y"][0],
#         "x1": zoom_in_ranges["x"][1],
#         "y1": zoom_in_ranges["y"][1],
#         "line": {
#             "width": 0.25,
#             "color": "black",
#             },
#         },
#     ]

## Generation 0

In [10]:
fig = dict(data=data_final, layout=layout)

if orca_avail:
    pio.write_image(fig, 'images/iro2_ml_convergence_gen_0.svg')

py.iplot(fig,
    filename=os.path.join(
        save_dir,
        "ml_convergence",
        "iro2",
        "iro2_ml_convergence_gen_0",
        ),
    )

/home/raul_surface/anaconda2/envs/test_bare/lib/python3.6/site-packages/IPython/core/display.py:689: UserWarning:

Consider using IPython.display.IFrame instead



## Generation 1

In [11]:
layout_i = copy.deepcopy(layout)

layout_i["xaxis"]["showticklabels"] = True

In [12]:
fig = dict(data=data_final2, layout=layout_i)

if orca_avail:
    pio.write_image(fig, 'images/iro2_ml_convergence_gen_1.svg')

py.iplot(fig,
    filename=os.path.join(
        save_dir,
        "ml_convergence",
        "iro2",
        "iro2_ml_convergence_gen_1",
        ),
    )

# Zoomed in plots

In [13]:
marker_size = 10

In [14]:
layout["xaxis"]["range"] = [0, 20]
layout["yaxis"]["range"] = [-0.8, -0.3]

layout["yaxis"]["dtick"] = 0.2
layout["width"] = (2. * 18.7 * 37.795275591) / 4.5
layout["height"] = (1. * 18.7 * 37.795275591) / 4.0

# Removing shapes
# layout.pop("shapes")

In [15]:
layout["yaxis"]["range"] = [-0.8102, -0.5747]

layout["yaxis"]["showticklabels"] = False

In [16]:
for i in data_final:
    i_dict = i.to_plotly_json()
    if "name" in i_dict.keys():
        if "energies" in i_dict["name"]:
            trace_j = i
            trace_j["marker"]["size"] = marker_size

fig = dict(data=data_final, layout=layout)

if orca_avail:
    pio.write_image(fig, 'images/iro2_ml_convergence_gen_0__zoomed_in.svg')
py.iplot(fig,
    filename=os.path.join(
        save_dir,
        "ml_convergence",
        "iro2",
        "iro2_ml_convergence_gen_0__zoomed_in",
        ),
    )

In [17]:
layout["yaxis"]["range"] = [-0.9, -0.5747]

In [18]:
for i in data_final2:
    i_dict = i.to_plotly_json()
    if "name" in i_dict.keys():
        if "energies" in i_dict["name"]:
            trace_j = i
            trace_j["marker"]["size"] = marker_size

fig = dict(data=data_final2, layout=layout)

if orca_avail:
    pio.write_image(fig, 'images/iro2_ml_convergence_gen_1__zoomed_in.svg')
py.iplot(fig,
    filename=os.path.join(
        save_dir,
        "ml_convergence",
        "iro2",
        "iro2_ml_convergence_gen_1__zoomed_in",
        ),
    )

# end_of_file